In [4]:
import torch
import torchvision
import os
import numpy as np
import torchvision.transforms as transforms
root = './data'
if not os.path.exists(root):
    os.mkdir(root)
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [54]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = torchvision.datasets.CIFAR10(root=root, train=False,transform=transform, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=15,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root=root, train=False, transform=transform, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=15,
                                         shuffle=False, num_workers=2)
classes = ('Aeroplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck')
print(len(trainset))
print(len(testset))

Files already downloaded and verified
Files already downloaded and verified
10000
10000


In [55]:
import numpy as np
"""import matplotlib.pyplot as plt
import numpy as np
def imshow(img):
    img = img/2 + 0.5     # unnormalize
    npimg = img.numpy()
    x = np.transpose(npimg, (1, 2, 0))
    plt.imshow(x)

dataiter = iter(trainloader)
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))"""

'import matplotlib.pyplot as plt\nimport numpy as np\ndef imshow(img):\n    img = img/2 + 0.5     # unnormalize\n    npimg = img.numpy()\n    x = np.transpose(npimg, (1, 2, 0))\n    plt.imshow(x)\n\ndataiter = iter(trainloader)\nimages, labels = dataiter.next()\nimshow(torchvision.utils.make_grid(images))'

In [56]:
import torch.nn as nn
import torch.nn.functional as F
class CIFAR10(nn.Module):
    def __init__(self):
        super(CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 3,1,(1,1))
        self.conv2 = nn.Conv2d(10, 30, 3)
        self.batch_1=nn.BatchNorm2d(30)
        self.fc1 = nn.Linear(7*7*30, 300)
        self.batch_2=nn.BatchNorm1d(300)
        self.fc2 = nn.Linear(300, 100)
        self.batch_3=nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.batch_1(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 7*7*30)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def name(self):
        return "CIFAR10"
        

In [61]:
import torch.optim as optim
model = CIFAR10()
model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.003)
criterion = nn.CrossEntropyLoss()

In [64]:
for epoch in range(45):
    # trainning
    ave_loss = 0
    for i,data in enumerate(trainloader,0):
        inputs, label = data
        inputs, label = inputs.cuda(), label.cuda()
        optimizer.zero_grad()
        out = model(inputs)
        loss = criterion(out,label)
        loss.backward()
        optimizer.step()
        ave_loss+= loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, ave_loss / 200))
            ave_loss = 0.0
print("Finished Training")


[1,   200] loss: 0.076
[1,   400] loss: 0.085
[1,   600] loss: 0.094
[2,   200] loss: 0.085
[2,   400] loss: 0.108
[2,   600] loss: 0.169
[3,   200] loss: 0.101
[3,   400] loss: 0.081
[3,   600] loss: 0.088
[4,   200] loss: 0.065
[4,   400] loss: 0.067
[4,   600] loss: 0.104
[5,   200] loss: 0.117
[5,   400] loss: 0.113
[5,   600] loss: 0.120
[6,   200] loss: 0.148
[6,   400] loss: 0.115
[6,   600] loss: 0.114
[7,   200] loss: 0.078
[7,   400] loss: 0.118
[7,   600] loss: 0.095
[8,   200] loss: 0.088
[8,   400] loss: 0.079
[8,   600] loss: 0.090
[9,   200] loss: 0.080
[9,   400] loss: 0.092
[9,   600] loss: 0.104
[10,   200] loss: 0.111
[10,   400] loss: 0.076
[10,   600] loss: 0.109
[11,   200] loss: 0.055
[11,   400] loss: 0.112
[11,   600] loss: 0.086
[12,   200] loss: 0.090
[12,   400] loss: 0.121
[12,   600] loss: 0.145
[13,   200] loss: 0.076
[13,   400] loss: 0.069
[13,   600] loss: 0.091
[14,   200] loss: 0.094
[14,   400] loss: 0.059
[14,   600] loss: 0.106
[15,   200] loss: 0

In [65]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        if use_cuda:
            images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(total)
print(correct)
print(100 * correct / total)

10000
9867
98.67
